In [ ]:
!apt-get update
!apt-get install -y build-essential nvidia-cuda-toolkit

# Tạo file mã nguồn CUDA

In [ ]:
%%writefile cuda_info.cu
#include <iostream>
#include <cuda_runtime.h>
using namespace std;
int main(int argc, char ** argv)
{
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);
    for (int dev = 0; dev < deviceCount; dev++)
    {
        cudaDeviceProp deviceProp;
        cudaGetDeviceProperties(&deviceProp, dev);
        if (dev == 0)
        {
            if (deviceProp.major == 9999 && deviceProp.minor == 9999)
            {
                cout << "No CUDA GPU has been detected" << endl;
                return -1;
            }
            else if (deviceCount == 1)
            {
                cout << "There is 1 device supporting CUDA" << endl;
            }
            else
            {
                cout << "There are " << deviceCount << " devices supporting CUDA" << endl;
            }
        }
        cout << "Device " << dev << " name: " << deviceProp.name << endl;
        cout << " Computational Capabilities: " << deviceProp.major << "." << deviceProp.minor << endl;
        cout << " Maximum global memory size: " << deviceProp.totalGlobalMem << endl;
        cout << " Maximum constant memory size: " << deviceProp.totalConstMem << endl;
        cout << " Maximum shared memory size per block: " << deviceProp.sharedMemPerBlock << endl;
        cout << " Maximum block dimensions: " << deviceProp.maxThreadsDim[0] << " x " << deviceProp.maxThreadsDim[1] << " x " << deviceProp.maxThreadsDim[2] << endl;
        cout << " Maximum grid dimensions: " << deviceProp.maxGridSize[0] << " x " << deviceProp.maxGridSize[1] << " x " << deviceProp.maxGridSize[2] << endl;
        cout << " Warp size: " << deviceProp.warpSize << endl;
    }
    cudaDeviceReset();
    return 0;
}


# Biên dịch mã CUDA

In [ ]:
!nvcc cuda_info.cu -o cuda_info

In [ ]:
!./cuda_info

# Matrix Multiplication

# Generate input file 

In [ ]:
import numpy as np

def generate_matrix_input_file(filename, m, k, n):
    """
    Generate an input file for matrix multiplication:
    - Matrix A of size m x k
    - Matrix B of size k x n
    - Resulting matrix C will be of size m x n
    """
    with open(filename, 'w') as f:
        # Write size of matrix A
        f.write(f"{m} {k}\n")

        # Generate and write matrix A (random integers between 0 and 100)
        A = np.random.randint(0, 101, size=(m, k))
        np.savetxt(f, A, fmt="%d", delimiter=" ")

        # Write size of matrix B
        f.write(f"{k} {n}\n")

        # Generate and write matrix B (random integers between 0 and 100)
        B = np.random.randint(0, 101, size=(k, n))
        np.savetxt(f, B, fmt="%d", delimiter=" ")

# Define matrix sizes
matrix_sizes = [100, 500, 1000, 2000, 5000, 8000, 10000]

# Generate input files
for size in matrix_sizes:
    filename = f"Matrix_{size}x{size}.INP"
    print(f"Generating {filename} ...")
    generate_matrix_input_file(filename, size, size, size)

print("\nAll matrix input files generated successfully!")

# CPU

In [ ]:
# Imports
import random
import numpy as np
import cudf
import cupy as cp
import cython
from numba import cuda, int32
from timeit import default_timer as timer
from scipy.sparse import lil_matrix
from pyspark.sql import SparkSession
from pyspark.mllib.linalg.distributed import *
import matplotlib.pyplot as plt

In [ ]:
%load_ext cython

In [ ]:
%%cython
import numpy as np
cimport numpy as np
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cdef np.ndarray[np.int32_t, ndim=2] _naive_dot(np.ndarray[np.int32_t, ndim=2] a, np.ndarray[np.int32_t, ndim=2] b):
    cdef np.ndarray[np.int32_t, ndim=2] c
    cdef int n, p, m
    cdef np.int32_t s
    if a.shape[1] != b.shape[0]:
        raise ValueError('shape not matched')
    n, p, m = a.shape[0], a.shape[1], b.shape[1]
    c = np.zeros((n, m), dtype=np.int32)
    for i in range(n):  # Use range instead of xrange for Python 3 compatibility
        for j in range(m):
            s = 0
            for k in range(p):
                s += a[i, k] * b[k, j]
            c[i, j] = s
    return c

def cpu_matrix_mul(a, b):
    return _naive_dot(a, b)

In [ ]:
import numpy as np
import time

def read_matrix_from_file(filename):
    """
    Đọc ma trận từ file và trả về hai ma trận NumPy.
    """
    with open(filename, "r") as f:
        # Đọc kích thước ma trận A
        m, k = map(int, f.readline().split())
        matrix_A = np.array([list(map(int, f.readline().split())) for _ in range(m)], dtype=np.int32)

        # Đọc kích thước ma trận B
        k_check, n = map(int, f.readline().split())
        assert k == k_check, "Kích thước ma trận không hợp lệ!"

        matrix_B = np.array([list(map(int, f.readline().split())) for _ in range(k)], dtype=np.int32)

    return matrix_A, matrix_B

# List of dataset filenames
datasets = [
    "Matrix_100x100.INP",
    "Matrix_500x500.INP",
    "Matrix_1000x1000.INP",
    "Matrix_2000x2000.INP",
    "Matrix_5000x5000.INP",
    "Matrix_8000x8000.INP",
    "Matrix_10000x10000.INP"
]

for dataset in datasets:
    A, B = read_matrix_from_file(dataset)
    
    # Measure execution time in milliseconds
    start_time = time.time()
    C = cpu_matrix_mul(A, B)
    end_time = time.time()
    
    execution_time_ms = (end_time - start_time) * 1000  # Convert to milliseconds
    print(f"Execution time for {dataset}: {execution_time_ms:.3f} ms")

# Cuda

In [ ]:
%%writefile matrixMultiCuda.cu
#include <iostream>
#include <fstream>       
#include <cuda_runtime.h>
#define INPUT_FILE "Matrix_8000x8000.INP"
#define OUTPUT_FILE "MATRIX_RESULT.OUT"
using namespace std;

#define THREADS 32
#define MAX_BLOCKS(size) ((size-1)/THREADS + 1)

// Matrix multiplication kernel
__global__ void matrixMultiply(float* A, float* B, float* C,
                              int numARows, int numAColumns,
                              int numBRows, int numBColumns,
                              int numCRows, int numCColumns) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    
    if ((row < numCRows) && (col < numCColumns)) {
        float value = 0;
        #pragma unroll
        for (int k = 0; k < numAColumns; ++k) {
            value += A[row * numAColumns + k] * B[k * numBColumns + col];
        }
        C[row * numCColumns + col] = value;
    }
}

int main(int argc, char** argv) {
    float *hostA, *hostB, *hostC;  // The A, B, and C (output) matrices
    float *deviceA, *deviceB, *deviceC;
    int numARows, numAColumns;    // dimensions of matrix A
    int numBRows, numBColumns;    // dimensions of matrix B
    int numCRows, numCColumns;    // dimensions of matrix C
    
    // Create CUDA events for timing
    cudaEvent_t start, stop, start_total, stop_total;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventCreate(&start_total);
    cudaEventCreate(&stop_total);
    float milliseconds, total_time;

    

    /* Importing data and creating memory on host */
    ifstream finp(INPUT_FILE);
    cudaEventRecord(start);
    
    // Read dimensions of matrix A
    finp >> numARows >> numAColumns;
    int byteSizeA = sizeof(float) * numARows * numAColumns;
    hostA = (float*)malloc(byteSizeA);
        
    // Read matrix A
    for(int i = 0; i < numARows; i++) {
        for(int j = 0; j < numAColumns; j++) {
            finp >> hostA[i * numAColumns + j];
        }
    }

    // Read dimensions of matrix B
    finp >> numBRows >> numBColumns;
    int byteSizeB = sizeof(float) * numBRows * numBColumns;
    hostB = (float*)malloc(byteSizeB);
    
    // Read matrix B
    for(int i = 0; i < numBRows; i++) {
        for(int j = 0; j < numBColumns; j++) {
            finp >> hostB[i * numBColumns + j];
        }
    }

    // Set dimensions of output matrix C
    numCRows = numARows;
    numCColumns = numBColumns;
    int byteSizeC = sizeof(float) * numCRows * numCColumns;
    hostC = (float*)malloc(byteSizeC);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    cout << "Time for importing data and creating host memory: " << milliseconds << " ms" << endl;
    
    cout << "Matrix Dimensions:" << endl;
    cout << "A: " << numARows << " x " << numAColumns << endl;
    cout << "B: " << numBRows << " x " << numBColumns << endl;
    cout << "C: " << numCRows << " x " << numCColumns << endl;

    /* Start measuring total time */
    cudaEventRecord(start_total);
    
    /* Allocating GPU memory */
    cudaEventRecord(start);
    cudaMalloc((void**)&deviceA, byteSizeA);
    cudaMalloc((void**)&deviceB, byteSizeB);
    cudaMalloc((void**)&deviceC, byteSizeC);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    cout << "Time for allocating GPU memory: " << milliseconds << " ms" << endl;

    /* Copying input memory to the GPU */
    cudaEventRecord(start);
    cudaMemcpy(deviceA, hostA, byteSizeA, cudaMemcpyHostToDevice);
    cudaMemcpy(deviceB, hostB, byteSizeB, cudaMemcpyHostToDevice);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    cout << "Time to copy data from CPU to GPU: " << milliseconds << " ms" << endl;

    /* Initialize the grid and block dimensions */
    dim3 blockDim(THREADS, THREADS);
    dim3 gridDim(MAX_BLOCKS(numCColumns), MAX_BLOCKS(numCRows));

    /* Launch the GPU Kernel */
    cudaEventRecord(start);
    matrixMultiply<<<gridDim, blockDim>>>(deviceA, deviceB, deviceC,
                                         numARows, numAColumns,
                                         numBRows, numBColumns,
                                         numCRows, numCColumns);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    cout << "Time for kernel computation on GPU: " << milliseconds << " ms" << endl;

    /* Copying output memory to the CPU */
    cudaEventRecord(start);
    cudaMemcpy(hostC, deviceC, byteSizeC, cudaMemcpyDeviceToHost);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    cout << "Time to copy data from GPU to CPU: " << milliseconds << " ms" << endl;

    /* End measuring total time */
    cudaEventRecord(stop_total);
    cudaEventSynchronize(stop_total);
    cudaEventElapsedTime(&total_time, start_total, stop_total);
    cout << "Total time: " << total_time << " ms" << endl;

    /* Output results */
    ofstream fout(OUTPUT_FILE);
    fout << numCRows << " " << numCColumns << endl;
    for(int i = 0; i < numCRows; i++) {
        for(int j = 0; j < numCColumns; j++) {
            fout << hostC[i * numCColumns + j] << " ";
        }
        fout << endl;
    }
    fout.close();

    /* Freeing GPU Memory */
    cudaFree(deviceA);
    cudaFree(deviceB);
    cudaFree(deviceC);

    /* Freeing Host Memory */
    free(hostA);
    free(hostB);
    free(hostC);

    /* Destroy CUDA events */
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaEventDestroy(start_total);
    cudaEventDestroy(stop_total);

    return 0;
}

In [ ]:
!nvcc matrixMultiCuda.cu -o matrixMultiCuda

In [ ]:
!./matrixMultiCuda

# Cuda Shared Memory

In [ ]:
%%writefile matrixMultiSharedMemory.cu
#include <iostream>
#include <fstream>       
#include <cuda_runtime.h>
#define INPUT_FILE "Matrix_8000x8000.INP"
#define OUTPUT_FILE "LAB02.OUT"
using namespace std;

#define TILE_WIDTH 32
#define THREADS TILE_WIDTH
#define MAX_BLOCKS(size) ((size-1)/THREADS + 1)

// Matrix multiplication kernel using shared memory
__global__ void matrixMultiplyShared(float* A, float* B, float* C,
                                    int numARows, int numAColumns,
                                    int numBRows, int numBColumns,
                                    int numCRows, int numCColumns) {
    __shared__ float sharedA[TILE_WIDTH][TILE_WIDTH];
    __shared__ float sharedB[TILE_WIDTH][TILE_WIDTH];
    
    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    
    // Global row and column indices
    int row = by * TILE_WIDTH + ty;
    int col = bx * TILE_WIDTH + tx;
    
    float value = 0;
    
    // Loop over tiles
    for (int t = 0; t < (numAColumns-1)/TILE_WIDTH + 1; ++t) {
        // Load data into shared memory
        if (row < numARows && (t*TILE_WIDTH + tx) < numAColumns)
            sharedA[ty][tx] = A[row * numAColumns + t * TILE_WIDTH + tx];
        else
            sharedA[ty][tx] = 0.0f;
            
        if ((t*TILE_WIDTH + ty) < numBRows && col < numBColumns)
            sharedB[ty][tx] = B[(t * TILE_WIDTH + ty) * numBColumns + col];
        else
            sharedB[ty][tx] = 0.0f;
            
        __syncthreads();
        
        // Compute partial dot product within tile
        #pragma unroll
        for (int k = 0; k < TILE_WIDTH; ++k) {
            value += sharedA[ty][k] * sharedB[k][tx];
        }
        __syncthreads();
    }
    
    // Write result
    if (row < numCRows && col < numCColumns) {
        C[row * numCColumns + col] = value;
    }
}

int main(int argc, char** argv) {
    float *hostA, *hostB, *hostC;
    float *deviceA, *deviceB, *deviceC;
    int numARows, numAColumns;
    int numBRows, numBColumns;
    int numCRows, numCColumns;
    
    cudaEvent_t start, stop, start_total, stop_total;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventCreate(&start_total);
    cudaEventCreate(&stop_total);
    float milliseconds, total_time;

    /* Importing data and creating memory on host */
    ifstream finp(INPUT_FILE);
    cudaEventRecord(start);
    
    finp >> numARows >> numAColumns;
    int byteSizeA = sizeof(float) * numARows * numAColumns;
    hostA = (float*)malloc(byteSizeA);
        
    for(int i = 0; i < numARows; i++) {
        for(int j = 0; j < numAColumns; j++) {
            finp >> hostA[i * numAColumns + j];
        }
    }

    finp >> numBRows >> numBColumns;
    int byteSizeB = sizeof(float) * numBRows * numBColumns;
    hostB = (float*)malloc(byteSizeB);
    
    for(int i = 0; i < numBRows; i++) {
        for(int j = 0; j < numBColumns; j++) {
            finp >> hostB[i * numBColumns + j];
        }
    }

    numCRows = numARows;
    numCColumns = numBColumns;
    int byteSizeC = sizeof(float) * numCRows * numCColumns;
    hostC = (float*)malloc(byteSizeC);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    cout << "Time for importing data and creating host memory: " << milliseconds << " ms" << endl;
    
    cout << "Matrix Dimensions:" << endl;
    cout << "A: " << numARows << " x " << numAColumns << endl;
    cout << "B: " << numBRows << " x " << numBColumns << endl;
    cout << "C: " << numCRows << " x " << numCColumns << endl;

    cudaEventRecord(start_total);
    
    cudaEventRecord(start);
    cudaMalloc((void**)&deviceA, byteSizeA);
    cudaMalloc((void**)&deviceB, byteSizeB);
    cudaMalloc((void**)&deviceC, byteSizeC);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    cout << "Time for allocating GPU memory: " << milliseconds << " ms" << endl;

    cudaEventRecord(start);
    cudaMemcpy(deviceA, hostA, byteSizeA, cudaMemcpyHostToDevice);
    cudaMemcpy(deviceB, hostB, byteSizeB, cudaMemcpyHostToDevice);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    cout << "Time to copy data from CPU to GPU: " << milliseconds << " ms" << endl;

    dim3 blockDim(TILE_WIDTH, TILE_WIDTH);
    dim3 gridDim(MAX_BLOCKS(numCColumns), MAX_BLOCKS(numCRows));

    cudaEventRecord(start);
    matrixMultiplyShared<<<gridDim, blockDim>>>(deviceA, deviceB, deviceC,
                                               numARows, numAColumns,
                                               numBRows, numBColumns,
                                               numCRows, numCColumns);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    cout << "Time for kernel computation on GPU: " << milliseconds << " ms" << endl;

    cudaEventRecord(start);
    cudaMemcpy(hostC, deviceC, byteSizeC, cudaMemcpyDeviceToHost);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    cudaEventElapsedTime(&milliseconds, start, stop);
    cout << "Time to copy data from GPU to CPU: " << milliseconds << " ms" << endl;

    cudaEventRecord(stop_total);
    cudaEventSynchronize(stop_total);
    cudaEventElapsedTime(&total_time, start_total, stop_total);
    cout << "Total time: " << total_time << " ms" << endl;

    ofstream fout(OUTPUT_FILE);
    fout << numCRows << " " << numCColumns << endl;
    for(int i = 0; i < numCRows; i++) {
        for(int j = 0; j < numCColumns; j++) {
            fout << hostC[i * numCColumns + j] << " ";
        }
        fout << endl;
    }
    fout.close();

    cudaFree(deviceA);
    cudaFree(deviceB);
    cudaFree(deviceC);

    free(hostA);
    free(hostB);
    free(hostC);

    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaEventDestroy(start_total);
    cudaEventDestroy(stop_total);

    return 0;
}

In [ ]:
!nvcc matrixMultiSharedMemory.cu -o matrixMultiSharedMemory

In [ ]:
!./matrixMultiSharedMemory

## Verison Cupy 

In [ ]:
import numpy as np
import cupy as cp
import time

# Define the CUDA kernel as a string
matrix_multiply_kernel = r'''
extern "C" __global__ void matrixMultiplyShared(
    const float* A, const float* B, float* C,
    int numARows, int numAColumns,
    int numBRows, int numBColumns,
    int numCRows, int numCColumns) {
    
    __shared__ float sharedA[32][32];
    __shared__ float sharedB[32][32];
    
    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    
    int row = by * 32 + ty;
    int col = bx * 32 + tx;
    
    float value = 0;
    
    for (int t = 0; t < (numAColumns-1)/32 + 1; ++t) {
        if (row < numARows && (t*32 + tx) < numAColumns)
            sharedA[ty][tx] = A[row * numAColumns + t * 32 + tx];
        else
            sharedA[ty][tx] = 0.0f;
            
        if ((t*32 + ty) < numBRows && col < numBColumns)
            sharedB[ty][tx] = B[(t * 32 + ty) * numBColumns + col];
        else
            sharedB[ty][tx] = 0.0f;
            
        __syncthreads();
        
        #pragma unroll
        for (int k = 0; k < 32; ++k) {
            value += sharedA[ty][k] * sharedB[k][tx];
        }
        __syncthreads();
    }
    
    if (row < numCRows && col < numCColumns) {
        C[row * numCColumns + col] = value;
    }
}
'''

def matrix_multiply(A, B):
    # Ensure inputs are CuPy arrays
    if isinstance(A, np.ndarray):
        A = cp.asarray(A)
    if isinstance(B, np.ndarray):
        B = cp.asarray(B)
    
    # Get matrix dimensions
    numARows, numAColumns = A.shape
    numBRows, numBColumns = B.shape
    
    # Validate matrix dimensions
    if numAColumns != numBRows:
        raise ValueError("Matrix dimensions do not match for multiplication")
    
    numCRows = numARows
    numCColumns = numBColumns
    
    # Allocate output matrix
    C = cp.zeros((numCRows, numCColumns), dtype=cp.float32)
    
    # Compile the CUDA kernel
    kernel = cp.RawKernel(matrix_multiply_kernel, 'matrixMultiplyShared')
    
    # Define grid and block dimensions
    threads_per_block = 32
    grid_x = (numCColumns - 1) // threads_per_block + 1
    grid_y = (numCRows - 1) // threads_per_block + 1
    
    # Time measurement
    start = time.time()
    
    # Launch kernel
    kernel(
        (grid_x, grid_y),
        (threads_per_block, threads_per_block),
        (A, B, C, numARows, numAColumns, numBRows, numBColumns, numCRows, numCColumns)
    )
    
    cp.cuda.stream.get_current_stream().synchronize()
    end = time.time()
    
    print(f"Kernel execution time: {(end - start) * 1000:.2f} ms")
    
    return C

# Example usage
if __name__ == "__main__":
    # Read input matrices from file
    with open("Matrix_8000x8000.INP", "r") as f:
        # Read matrix A
        numARows, numAColumns = map(int, f.readline().split())
        A = np.zeros((numARows, numAColumns), dtype=np.float32)
        for i in range(numARows):
            A[i] = list(map(float, f.readline().split()))
        
        # Read matrix B
        numBRows, numBColumns = map(int, f.readline().split())
        B = np.zeros((numBRows, numBColumns), dtype=np.float32)
        for i in range(numBRows):
            B[i] = list(map(float, f.readline().split()))
    
    print(f"Matrix A shape: {A.shape}")
    print(f"Matrix B shape: {B.shape}")
    
    # Perform matrix multiplication
    start_total = time.time()
    
    # Transfer to GPU
    start = time.time()
    A_gpu = cp.asarray(A)
    B_gpu = cp.asarray(B)
    cp.cuda.stream.get_current_stream().synchronize()
    end = time.time()
    print(f"Time to copy data to GPU: {(end - start) * 1000:.3f} ms")
    
    # Multiply matrices
    C_gpu = matrix_multiply(A_gpu, B_gpu)
    
    # Transfer result back to CPU
    start = time.time()
    C = cp.asnumpy(C_gpu)
    cp.cuda.stream.get_current_stream().synchronize()
    end = time.time()
    print(f"Time to copy data from GPU: {(end - start) * 1000:.3f} ms")
    
    end_total = time.time()
    print(f"Total time: {(end_total - start_total) * 1000:.3f} ms")

    # Tính kết quả trên CPU bằng NumPy
    C_numpy = np.dot(A, B)

    # So sánh kết quả giữa GPU và CPU
    if np.allclose(C, C_numpy, atol=1e-5):
        print("Kết quả chính xác! GPU và CPU cho cùng một kết quả.")
    else:
        print("Kết quả không khớp! Kiểm tra lại phép nhân ma trận hoặc kernel CUDA.")
    
    # Save result to file
    with open("LAB02.OUT", "w") as f:
        f.write(f"{C.shape[0]} {C.shape[1]}\n")
        for row in C:
            f.write(" ".join(map(str, row)) + "\n")

In [ ]:
import numpy as np
import cupy as cp
import time

# Define the CUDA kernel as a string
matrix_multiply_kernel = r'''
extern "C" __global__ void matrixMultiplyShared(
    const float* A, const float* B, float* C,
    int numARows, int numAColumns,
    int numBRows, int numBColumns,
    int numCRows, int numCColumns) {
    
    __shared__ float sharedA[32][32];
    __shared__ float sharedB[32][32];
    
    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    
    int row = by * 32 + ty;
    int col = bx * 32 + tx;
    
    float value = 0;
    
    for (int t = 0; t < (numAColumns-1)/32 + 1; ++t) {
        if (row < numARows && (t*32 + tx) < numAColumns)
            sharedA[ty][tx] = A[row * numAColumns + t * 32 + tx];
        else
            sharedA[ty][tx] = 0.0f;
            
        if ((t*32 + ty) < numBRows && col < numBColumns)
            sharedB[ty][tx] = B[(t * 32 + ty) * numBColumns + col];
        else
            sharedB[ty][tx] = 0.0f;
            
        __syncthreads();
        
        #pragma unroll
        for (int k = 0; k < 32; ++k) {
            value += sharedA[ty][k] * sharedB[k][tx];
        }
        __syncthreads();
    }
    
    if (row < numCRows && col < numCColumns) {
        C[row * numCColumns + col] = value;
    }
}
'''

def matrix_multiply(A, B):
    # Ensure inputs are CuPy arrays
    if isinstance(A, np.ndarray):
        A = cp.asarray(A)
    if isinstance(B, np.ndarray):
        B = cp.asarray(B)
    
    # Get matrix dimensions
    numARows, numAColumns = A.shape
    numBRows, numBColumns = B.shape
    
    # Validate matrix dimensions
    if numAColumns != numBRows:
        raise ValueError("Matrix dimensions do not match for multiplication")
    
    numCRows = numARows
    numCColumns = numBColumns
    
    # Allocate output matrix (Measure time)
    start_alloc = time.time()
    C = cp.zeros((numCRows, numCColumns), dtype=cp.float32)
    cp.cuda.stream.get_current_stream().synchronize()
    end_alloc = time.time()
    
    # Compile the CUDA kernel
    kernel = cp.RawKernel(matrix_multiply_kernel, 'matrixMultiplyShared')
    
    # Define grid and block dimensions
    threads_per_block = 32
    grid_x = (numCColumns - 1) // threads_per_block + 1
    grid_y = (numCRows - 1) // threads_per_block + 1
    
    # Kernel computation time
    start_kernel = time.time()
    kernel(
        (grid_x, grid_y),
        (threads_per_block, threads_per_block),
        (A, B, C, numARows, numAColumns, numBRows, numBColumns, numCRows, numCColumns)
    )
    cp.cuda.stream.get_current_stream().synchronize()
    end_kernel = time.time()
    
    print(f"Time for allocating GPU memory: {(end_alloc - start_alloc) * 1000:.2f} ms")
    print(f"Kernel execution time: {(end_kernel - start_kernel) * 1000:.2f} ms")
    
    return C

# Example usage
if __name__ == "__main__":
    # Read input matrices from file
    with open("LAB02_large.INP", "r") as f:
        # Read matrix A
        numARows, numAColumns = map(int, f.readline().split())
        A = np.zeros((numARows, numAColumns), dtype=np.float32)
        for i in range(numARows):
            A[i] = list(map(float, f.readline().split()))
        
        # Read matrix B
        numBRows, numBColumns = map(int, f.readline().split())
        B = np.zeros((numBRows, numBColumns), dtype=np.float32)
        for i in range(numBRows):
            B[i] = list(map(float, f.readline().split()))
    
    print(f"Matrix A shape: {A.shape}")
    print(f"Matrix B shape: {B.shape}")
    
    # Total execution time measurement
    start_total = time.time()
    
    # Copy data to GPU
    start_copy_to_gpu = time.time()
    A_gpu = cp.asarray(A)
    B_gpu = cp.asarray(B)
    cp.cuda.stream.get_current_stream().synchronize()
    end_copy_to_gpu = time.time()
    
    print(f"Time to copy data to GPU: {(end_copy_to_gpu - start_copy_to_gpu) * 1000:.2f} ms")
    
    # Multiply matrices
    C_gpu = matrix_multiply(A_gpu, B_gpu)
    
    # Copy result back to CPU
    start_copy_from_gpu = time.time()
    C = cp.asnumpy(C_gpu)
    cp.cuda.stream.get_current_stream().synchronize()
    end_copy_from_gpu = time.time()
    
    print(f"Time to copy data from GPU: {(end_copy_from_gpu - start_copy_from_gpu) * 1000:.2f} ms")
    
    end_total = time.time()
    print(f"Total execution time: {(end_total - start_total) * 1000:.2f} ms")

    # Verify correctness with NumPy
    C_numpy = np.dot(A, B)

    if np.allclose(C, C_numpy, atol=1e-5):
        print("✅ GPU and CPU results match!")
    else:
        print("❌ GPU and CPU results do not match. Please check.")

    # Save result to file
    with open("LAB02.OUT", "w") as f:
        f.write(f"{C.shape[0]} {C.shape[1]}\n")
        for row in C:
            f.write(" ".join(map(str, row)) + "\n")

## Rapids trên Kernel Cupy

In [ ]:
import cupy as cp
import cudf
import time

# CUDA Kernel for Matrix Multiplication
matrix_mult_kernel = cp.RawKernel(r'''
extern "C" __global__ void matmul(
    float* A, float* B, float* C, int N, int M, int P) {
    
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < P) {
        float sum = 0;
        for (int k = 0; k < M; k++) {
            sum += A[row * M + k] * B[k * P + col];
        }
        C[row * P + col] = sum;
    }
}
''', 'matmul')

# Custom Matrix Multiplication Function
def matmul(self, other):
    if not isinstance(other, cudf.DataFrame):
        raise ValueError("Matrix multiplication requires another cuDF DataFrame")

    # Start total execution time measurement
    total_start = time.time()

    # 1. Time for Memory Allocation (GPU)
    start_alloc = time.time()
    A_gpu = cp.empty(self.shape, dtype=cp.float32)
    B_gpu = cp.empty(other.shape, dtype=cp.float32)
    C_gpu = cp.empty((self.shape[0], other.shape[1]), dtype=cp.float32)
    end_alloc = time.time()

    # 2. Time for CPU → GPU Copy
    start_copy_h2d = time.time()
    A_gpu[:] = cp.array(self.values, dtype=cp.float32)
    B_gpu[:] = cp.array(other.values, dtype=cp.float32)
    end_copy_h2d = time.time()

    # Matrix dimensions
    N, M = A_gpu.shape
    M_B, P = B_gpu.shape
    if M != M_B:
        raise ValueError("Matrix dimensions do not match for multiplication")

    # 3. Time for Kernel Execution
    threads_per_block = (16, 16)
    blocks_per_grid = ((P + 15) // 16, (N + 15) // 16)
    start_kernel = time.time()
    matrix_mult_kernel(blocks_per_grid, threads_per_block, (A_gpu, B_gpu, C_gpu, N, M, P))
    cp.cuda.Device(0).synchronize()  # Ensure GPU computation finishes
    end_kernel = time.time()

    # 4. Time for GPU → CPU Copy
    start_copy_d2h = time.time()
    result = C_gpu.get()
    end_copy_d2h = time.time()

    # End total execution time measurement
    total_end = time.time()

    # Convert back to cuDF DataFrame
    result_df = cudf.DataFrame(result)

    # Convert all times to milliseconds
    alloc_time = (end_alloc - start_alloc) * 1000
    copy_h2d_time = (end_copy_h2d - start_copy_h2d) * 1000
    kernel_time = (end_kernel - start_kernel) * 1000
    copy_d2h_time = (end_copy_d2h - start_copy_d2h) * 1000
    total_time = (total_end - total_start) * 1000

    # Print Timing Information
    print(f"Memory Allocation Time: {alloc_time:.3f} ms")
    print(f"CPU → GPU Copy Time: {copy_h2d_time:.3f} ms")
    print(f"Kernel Execution Time: {kernel_time:.3f} ms")
    print(f"GPU → CPU Copy Time: {copy_d2h_time:.3f} ms")
    print(f"Total Execution Time: {total_time:.3f} ms")
    
    return result_df

# Monkey-patch cuDF DataFrame to add the custom method
cudf.DataFrame.matmul = matmul

# Function to read matrix from file
def read_matrix_from_file(filename):
    """
    Đọc ma trận từ file và trả về 2 ma trận A, B dưới dạng cuDF DataFrame.
    """
    with open(filename, "r") as f:
        # Read matrix A dimensions
        m, k = map(int, f.readline().split())
        matrix_A = [list(map(float, f.readline().split())) for _ in range(m)]
        
        # Read matrix B dimensions
        k_check, n = map(int, f.readline().split())
        assert k == k_check, "Matrix dimensions do not match!"
        
        matrix_B = [list(map(float, f.readline().split())) for _ in range(k)]
    
    # Convert to cuDF DataFrame
    df_A = cudf.DataFrame(matrix_A)
    df_B = cudf.DataFrame(matrix_B)
    
    return df_A, df_B, m, n

# Function to execute matrix multiplication and print execution time
def run_rapids_matrix_multiplication(filename):
    print(f"\nProcessing {filename} ...")
    A_df, B_df, _, _ = read_matrix_from_file(filename)

    # Perform matrix multiplication using RAPIDS
    result_df = A_df.matmul(B_df)

    print("Result (First 5 Rows):")
    print(result_df.head())

# List of test datasets
datasets = [
    "Matrix_100x100.INP",
    "Matrix_500x500.INP",
    "Matrix_1000x1000.INP",
    "Matrix_2000x2000.INP",
    "Matrix_5000x5000.INP",
    "Matrix_8000x8000.INP",
    "Matrix_10000x10000.INP"
]

# Run on all datasets
for dataset in datasets:
    run_rapids_matrix_multiplication(dataset)


## 